In [3]:
import requests
import urllib
from bs4 import BeautifulSoup
import csv
import pprint
import json
import codecs

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import chromedriver_binary

In [4]:
json_open = open("./data/voice_actors.json")
json_load = json.load(json_open)

In [22]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(30)

for actor_index, person in enumerate(json_load):
    print(f"actor:{actor_index} {person}")
    for i in range(len(json_load[person])):
        character = str(json_load[person][i]['character'])
        title = json_load[person][i]['title']
        print(f"actor:{actor_index} {person}, character: {character}, title: {title}")
    #url = requests.get("https://search.yahoo.co.jp/image/search?p="+str(chara)+"&aq=-1&ai=71637427-3001-4372-9c9e-ab4f345256eb&ts=4502&sfp=1&ei=UTF-8&fr=sfp_as#c4e1721a389b3f704b59332677756c92580cefdbd461a814ab1cd09f8c289c61")    
        url = requests.get(f"https://www.google.com/search?hl=jp&q={character} {title}&btnG=Google+Search&tbs=0&safe=off&tbm=isch")
        soup = BeautifulSoup(url.content, "html.parser")
        url.close()
        title = soup.title.string
        links = soup.find_all("img")
        num = 1
        if(len(links) == 1):
            num = 0
        elif (len(links) == 0):
            continue
        link = links[num].get("src")
        json_load[person][i]['img_url'] = link
        
        url = f"https://www.google.com/search?hl=jp&q={character} {title}&btnG=Google+Search&tbs=0&safe=off"
        driver.get(url)
        WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located)
        html = driver.page_source.encode('utf-8')
        soup = BeautifulSoup(html, "html.parser")
        if soup.find(attrs={"id":"result-stats"}):
            hit_num =  soup.find(attrs={"id":"result-stats"}).contents[0].split()[1]
            json_load[person][i]['hit_num'] = hit_num

    file_path = "./data/voice_actors_with_img_url_and_hitnum.json"
    outfile = codecs.open(file_path, 'a','utf-8')
    json.dump(json_load, outfile, ensure_ascii=False)

actor:0 麻倉もも
actor:0 麻倉もも, character: 鈴村優, title: IDOLY PRIDE


KeyboardInterrupt: 